In [1]:
# Data manipulation
import pandas as pd
import numpy as np
import numpy as np
from time import time, sleep

# Ploting and image operations
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
from PIL import Image
from PIL import ImageEnhance
import PIL.ImageOps

# Sklearn
from sklearn import preprocessing, metrics

# Keras libraries
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Conv3D, MaxPooling3D, GlobalAveragePooling2D
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.optimizers import SGD,Adam
from keras import regularizers
from keras.callbacks import ReduceLROnPlateau
from keras.applications.inception_v3 import InceptionV3
from keras import backend as K

# Other
import tensorflow as tf
import os
import sys
import random
import warnings
from six.moves import cPickle as pickle
from __future__ import print_function

Using TensorFlow backend.


In [2]:
np.random.seed(123) # Set random seed

### data preparation

Previously, we have downloaded posters for about 69,000 movies from TMDb in w154 resolution (154x231 pixels) and `.jpg` format. Here, we use the Python Imaging Library (PIL) to pre-process these `.jpg` images into matrices of pixel values. We randomly picked 1000 samples from each genre (19 genres totally) and convert them into  into matrices of dimensions 154x154x3. Those matrice will be used as training and validation set.


In [3]:
genres = pd.read_csv('/home/ubuntu/data/clean_data/genres_70k_new.txt')
genres = genres.drop('TVMovie', 1)

In [4]:
tmdb_index_dict={}
for i in range(genres.shape[0]):
    tmdb_index_dict[genres.iloc[i,0]]=i

In [5]:
DATA_PATH = "/data/poster_genre/"
SAVE_PATH="/data/poster_genre_resize/"
VALIDATION_PERCENT = .2
TEST_PERCENT = .2
IMAGE_WIDTH = 154
IMAGE_HEIGHT=154
IMAGE_SIZE=154
NUM_CHANNELS = 3 # RGB channels
PIXEL_DEPTH = 255.0
NUM_GENRES = 19
NUM_ROWS=1000
PARTITION_TEST = False

In [6]:
def to_rgb(im):
    w, h = im.shape
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, :] = im[:, :, np.newaxis]
    return ret
def read_image_from_file(file_path):
    img = Image.open(file_path).convert('RGB')
    img = img.resize((IMAGE_WIDTH,IMAGE_HEIGHT), Image.ANTIALIAS) #downsample image
    pixel_values = np.array(img.getdata())
    return np.reshape(pixel_values, [IMAGE_WIDTH,IMAGE_HEIGHT, NUM_CHANNELS])
def make_dataset_arrays(num_rows=NUM_ROWS,label=2):
    data = np.ndarray((num_rows, IMAGE_WIDTH,IMAGE_HEIGHT, NUM_CHANNELS), dtype=np.float32)
    labels = np.ndarray((num_rows, NUM_GENRES), dtype=np.int32)
    if label==3:
        single_label=np.ndarray((num_rows, NUM_GENRES), dtype=np.int32)
        return data, labels, single_label
    else:
        return data, labels
def scale_pixel_values(dataset):
    return (dataset - PIXEL_DEPTH / 2.0) / PIXEL_DEPTH

In [7]:
directory_list = list()
for root, dirs, files in os.walk(DATA_PATH, topdown=False):
    for name in dirs:
        directory_list.append(os.path.join(root, name))
print(directory_list)

['/data/poster_genre/Comedy', '/data/poster_genre/War', '/data/poster_genre/Crime', '/data/poster_genre/Adventure', '/data/poster_genre/Foreign', '/data/poster_genre/History', '/data/poster_genre/Fantasy', '/data/poster_genre/Western', '/data/poster_genre/Mystery', '/data/poster_genre/Romance', '/data/poster_genre/Animation', '/data/poster_genre/Music', '/data/poster_genre/Drama', '/data/poster_genre/Science Fiction', '/data/poster_genre/Documentary', '/data/poster_genre/Family', '/data/poster_genre/Action', '/data/poster_genre/Horror', '/data/poster_genre/Thriller']


In [8]:
genres.columns.tolist()[1:]

['Adventure',
 'Fantasy',
 'Animation',
 'Drama',
 'Horror',
 'Action',
 'Comedy',
 'History',
 'Western',
 'Thriller',
 'Crime',
 'Documentary',
 'Science Fiction',
 'Mystery',
 'Music',
 'Romance',
 'Family',
 'War',
 'Foreign']

In [9]:
label_order=genres.columns.tolist()[1:]
label_dict={}
for i in range(len(label_order)):
    label_dict[label_order[i]]=i
for d in directory_list:
    genre_name=d.split('/')[3]
    file_list = []
    for (dirpath, dirnames, filenames) in os.walk(d):
        file_list.extend(filenames)    
    print(genre_name,len(file_list))

Comedy 20844
War 1829
Crime 6117
Adventure 4801
Foreign 4984
History 1799
Fantasy 3221
Western 1830
Mystery 3295
Romance 9599
Animation 3967
Music 3208
Drama 29272
Science Fiction 4170
Documentary 6250
Family 4698
Action 10524
Horror 7570
Thriller 10391


In [46]:
for d in directory_list[1:2]:
    genre_name=d.split('/')[3]
    num_train=0
    train_data, train_labels, train_single_label = make_dataset_arrays(label=3)
    file_list = []
    for (dirpath, dirnames, filenames) in os.walk(d):
        file_list.extend(filenames)    
    samples=random.sample(file_list,10)
    for sample in samples:
        image=read_image_from_file(d+"/"+sample)
        tmdb_id=int(sample.split(".")[0])
        index=tmdb_index_dict[tmdb_id]
        labels=genres.iloc[index,1:]
        train_data[num_train]=image
        train_labels[num_train]=labels
        ## make single-class label
        single_label=np.zeros(NUM_GENRES)
        single_label[label_dict[genre_name]]=1
        train_single_label[num_train]=single_label
        num_train +=1


[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]]


In [52]:
#label_order=genres.columns.tolist()[1:]
#label_dict={}
#for i in range(len(label_order)):
    #label_dict[label_order[i]]=i
for d in directory_list:
    startTime = time()
    num_train = 0
    genre_name=d.split('/')[3]
    train_data, train_labels, train_single_label = make_dataset_arrays(label=3)
    file_list = []
    for (dirpath, dirnames, filenames) in os.walk(d):
        file_list.extend(filenames)    
    samples=random.sample(file_list,NUM_ROWS)#NUM_ROWS
    for sample in samples:
        image=read_image_from_file(d+"/"+sample)
        tmdb_id=int(sample.split(".")[0])
        index=tmdb_index_dict[tmdb_id]
        labels=genres.iloc[index,1:]
        train_data[num_train]=image
        train_labels[num_train]=labels
        ## make single-class label
        single_label=np.zeros(NUM_GENRES)
        single_label[label_dict[genre_name]]=1
        train_single_label[num_train]=single_label
        num_train +=1
    #train_data = scale_pixel_values(train_data)
    pickle_file = SAVE_PATH+genre_name+'_data.pickle'
    save = {
        'train_data': train_data,
        'train_labels': train_labels,
        'single_label': train_single_label
    }
    pickle.dump(save,open(pickle_file, 'wb'))
    endTime = time()-startTime
    print(genre_name,endTime)

Comedy 77.32352590560913
War 76.26311612129211
Crime 86.79342079162598
Adventure 64.31107664108276
Foreign 67.91265320777893
History 55.410622358322144
Fantasy 52.43929839134216
Western 41.751176834106445
Mystery 56.86282134056091
Romance 27.772007703781128
Animation 64.30848813056946
Music 24.333733081817627
Drama 67.78369450569153
Science Fiction 23.761247396469116
Documentary 56.50035738945007
Family 22.92879581451416
Action 59.71014142036438
Horror 22.35083508491516
Thriller 60.032227993011475


In [10]:
input_shape = (IMAGE_WIDTH,IMAGE_HEIGHT, 3)
#sample_index=random.sample(range(69), num_batch)
sample_index=np.arange(30)
train_index=label_order
test_index=sample_index[10:20]
val_index=sample_index[20:]
num_test=len(test_index)*NUM_ROWS
num_train=len(train_index)*NUM_ROWS
num_val=len(val_index)*NUM_ROWS
train_X=np.zeros((num_train,IMAGE_WIDTH,IMAGE_HEIGHT, 3))
train_Y1=np.zeros((num_train,NUM_GENRES))
train_Y2=np.zeros((num_train,NUM_GENRES))
test_X=np.zeros((num_test,IMAGE_WIDTH,IMAGE_HEIGHT, 3))
test_Y=np.zeros((num_test,NUM_GENRES))
val_X=np.zeros((num_val,IMAGE_WIDTH,IMAGE_HEIGHT, 3))
val_Y=np.zeros((num_val,NUM_GENRES))

In [11]:
input_shape = (IMAGE_WIDTH,IMAGE_HEIGHT, 3)
# smaller batch size means noisier gradient, but more updates per epoch
batch_size = 100
# this is fixed, we have 19 output classes
num_classes = NUM_GENRES
classes = NUM_GENRES
# sample batch
num_batch = 15

In [12]:
# Training data
train_index=label_order
num_train=len(train_index)*NUM_ROWS
train_X=np.zeros((num_train,IMAGE_WIDTH,IMAGE_HEIGHT, 3))
train_Y1=np.zeros((num_train,NUM_GENRES))
train_Y2=np.zeros((num_train,NUM_GENRES))
for i in range(len(train_index)):
    file_name=str(train_index[i])+"_data.pickle"
    pickle_file=SAVE_PATH+file_name
    with open(pickle_file, 'rb') as f:
        save = pickle.load(f)
        train_X[i*NUM_ROWS:(i+1)*NUM_ROWS] = save['train_data']
        train_Y1[i*NUM_ROWS:(i+1)*NUM_ROWS] = save['train_labels']
        train_Y2[i*NUM_ROWS:(i+1)*NUM_ROWS] = save['single_label']

In [13]:
datagen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

### multi-class classification

Here, we are using previous structure. But instead of multi-label classification, we tried multi-class classification.

In [25]:
model = Sequential()
# input: 154x154 images with 3 channels -> (154, 154, 3) tensors.
# this applies 64 convolution filters of size 3x3 each.
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_GENRES,activation='softmax'))

# prints out a summary of the model architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 150, 150, 64)      4864      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 148, 148, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 72, 72, 32)        18464     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 70, 70, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 35, 35, 32)        0         
__________

In [26]:
adam=Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#sgd = SGD(lr=0.1, momentum=0.9)
#early_stopping = EarlyStopping(monitor='val_loss',patience=2)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=1, min_lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [27]:
c = np.c_[train_X.reshape(len(train_X), -1), train_Y2.reshape(len(train_Y2), -1)]
np.random.shuffle(c)
x_train = c[:, :train_X.size//len(train_X)].reshape(train_X.shape)
y_train = c[:, train_X.size//len(train_X):].reshape(train_Y2.shape)

In [28]:
y_train.shape

(19000, 19)

In [29]:
epochs = 10
history = model.fit(x_train, y_train, batch_size=batch_size, 
                    epochs=epochs,
                    validation_split=0.2,
                    callbacks=[reduce_lr])

Train on 15200 samples, validate on 3800 samples
Epoch 1/10
15200/15200 [==============================] - 127s - loss: 15.2781 - acc: 0.0520 - val_loss: 15.2783 - val_acc: 0.0521

In [30]:
model.save_weights("/data/model/model_multiclass_19000.h5")
print("Saved model to disk")

Saved model to disk


In [78]:
pred_val_Y = model.predict(x_train)

In [80]:
# Drop tmdb_id
genres = genres.drop('tmdb_id', 1)
acc_baseline = np.mean(pd.DataFrame(pred_val_Y, columns=genres.columns) == pd.DataFrame(y_train, columns=genres.columns))
acc_baseline

Adventure          0.947368
Fantasy            0.947368
Animation          0.947368
Drama              0.947368
Horror             0.947368
Action             0.947368
Comedy             0.947368
History            0.947368
Western            0.947368
Thriller           0.947368
Crime              0.065105
Documentary        0.947368
Science Fiction    0.934053
Mystery            0.947368
Music              0.947368
Romance            0.947368
Family             0.947368
War                0.947368
Foreign            0.947368
dtype: float64

In [86]:
print("Overall accuracy CNN(from scratch)", round(acc_baseline.sum() / 19 * 100, 2), "%\n")

Overall accuracy CNN(from scratch) 90.02 %



In [87]:
warnings.filterwarnings('ignore')
# F1 score
print("Precision CNN(from scratch)", round(metrics.precision_score(y_train, pred_val_Y, average='weighted'), 3))
print("Recall CNN(from scratch)", round(metrics.recall_score(y_train, pred_val_Y, average='weighted'), 3))
print("F1 score CNN(from scratch)", round(metrics.f1_score(y_train, pred_val_Y, average='weighted'), 3))

Precision CNN(from scratch) 0.005
Recall CNN(from scratch) 0.052
F1 score CNN(from scratch) 0.006


### Multi-label classification

In [15]:
model = Sequential()
# input: 154x154 images with 3 channels -> (154, 154, 3) tensors.
# this applies 64 convolution filters of size 3x3 each.
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_GENRES,activation='sigmoid'))

# prints out a summary of the model architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 150, 150, 64)      4864      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 148, 148, 64)      36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 32)        18464     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 70, 70, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 35, 35, 32)        0         
__________

In [16]:
adam=Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
#sgd = SGD(lr=0.1, momentum=0.9)
#early_stopping = EarlyStopping(monitor='val_loss',patience=2)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
              patience=1, min_lr=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [17]:
c = np.c_[train_X.reshape(len(train_X), -1), train_Y1.reshape(len(train_Y1), -1)]
np.random.shuffle(c)
x_train = c[:, :train_X.size//len(train_X)].reshape(train_X.shape)
y_train = c[:, train_X.size//len(train_X):].reshape(train_Y1.shape)

In [18]:
epochs = 10
history = model.fit(x_train, y_train, batch_size=batch_size, 
                    epochs=epochs,
                    validation_split=0.2,
                    callbacks=[reduce_lr],
                    class_weight='Auto')

Train on 15200 samples, validate on 3800 samples
Epoch 1/10
15200/15200 [==============================] - 133s - loss: 2.7342 - acc: 0.8288 - val_loss: 2.2376 - val_acc: 0.8612

In [19]:
pred_val_Y = model.predict(x_train[15200:])

In [20]:
y_val=y_train[15200:]

In [21]:
warnings.filterwarnings('ignore')
# F1 score
print("Precision CNN(from scratch)", round(metrics.precision_score(y_val, pred_val_Y, average='weighted'), 3))
print("Recall CNN(from scratch)", round(metrics.recall_score(y_val, pred_val_Y, average='weighted'), 3))
print("F1 score CNN(from scratch)", round(metrics.f1_score(y_val, pred_val_Y, average='weighted'), 3))

Precision CNN(from scratch) 0.0
Recall CNN(from scratch) 0.0
F1 score CNN(from scratch) 0.0


In [24]:
model.save_weights("/data/model/model_multilable_19000.h5")
print("Saved model to disk")

Saved model to disk


### train binary model (one-vs-rest)

In [17]:
train_index

['Adventure',
 'Fantasy',
 'Animation',
 'Drama',
 'Horror',
 'Action',
 'Comedy',
 'History',
 'Western',
 'Thriller',
 'Crime',
 'Documentary',
 'Science Fiction',
 'Mystery',
 'Music',
 'Romance',
 'Family',
 'War',
 'Foreign']

In [11]:
input_shape = (IMAGE_WIDTH,IMAGE_HEIGHT, 3)
# smaller batch size means noisier gradient, but more updates per epoch
batch_size = 100
# this is fixed, we have 19 output classes
num_classes = NUM_GENRES
classes = NUM_GENRES
# sample batch
num_batch = 15

In [21]:
# Training data
train_index=label_order
num_train=len(train_index)*NUM_ROWS
train_X=np.zeros((num_train,IMAGE_WIDTH,IMAGE_HEIGHT, 3))
train_Y1=np.zeros((num_train,NUM_GENRES))
train_Y2=np.zeros((num_train,NUM_GENRES))
for i in range(len(train_index)):
    file_name=str(train_index[i])+"_data.pickle"
    pickle_file=SAVE_PATH+file_name
    with open(pickle_file, 'rb') as f:
        save = pickle.load(f)
        train_X[i*NUM_ROWS:(i+1)*NUM_ROWS] = save['train_data']
        train_Y1[i*NUM_ROWS:(i+1)*NUM_ROWS] = save['train_labels']
        train_Y2[i*NUM_ROWS:(i+1)*NUM_ROWS] = save['single_label']

In [20]:
train_X.shape

(19000, 154, 154, 3)

In [25]:
neg_index=random.sample(np.concatenate((np.arange(0,2000),np.arange(3000,19000))).tolist(),NUM_ROWS)

In [27]:
## train binary classifier for Animation
## random pick negative samples
neg_index=random.sample(np.concatenate((np.arange(0,2000),np.arange(3000,19000))).tolist(),NUM_ROWS)
#neg_index=random.sample(range(1000,19000),NUM_ROWS)
pos_index=np.arange(2000,3000).tolist()
pos_index.extend(neg_index)
adventure_X=train_X[pos_index]
adventure_Y=np.zeros((2*NUM_ROWS,2))
#adventure_Y=np.zeros((NUM_ROWS,2))
adventure_Y[:NUM_ROWS,0]=1
adventure_Y[NUM_ROWS:,1]=1

In [28]:
x_train=scale_pixel_values(adventure_X)
y_train=adventure_Y

In [29]:
c = np.c_[adventure_X.reshape(len(adventure_X), -1), adventure_Y.reshape(len(adventure_Y), -1)]
np.random.shuffle(c)
x_train = c[:, :adventure_X.size//len(adventure_X)].reshape(adventure_X.shape)
y_train = c[:, adventure_X.size//len(adventure_X):].reshape(adventure_Y.shape)
x_train=scale_pixel_values(x_train)

In [30]:
model = Sequential()
# input: 154x154 images with 3 channels -> (154, 154, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='sigmoid'))

# prints out a summary of the model architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 64)      4864      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 148, 148, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 72, 72, 32)        18464     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 70, 70, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 32)        0         
__________

In [31]:
adam=Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
#sgd = SGD(lr=0.001, momentum=0.9)
early_stopping = EarlyStopping(monitor='val_loss',patience=2)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=1, min_lr=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [32]:
epochs = 10
history = model.fit(x_train, y_train, batch_size=100, 
                    epochs=epochs,
                    validation_split=0.3,
                    callbacks=[early_stopping])

Train on 1400 samples, validate on 600 samples
Epoch 1/10
1400/1400 [==============================] - 18s - loss: 1.3159 - acc: 0.4939 - val_loss: 0.6931 - val_acc: 0.5067

In [33]:
model.save_weights("/data/model/model_animation.h5")
print("Saved model to disk")

Saved model to disk


In [27]:
model.load_weights("/data/model/model_adventure.h5")